In [7]:
import mysql.connector #type:ignore
import json

In [29]:
path = r"C:\Users\rando\OneDrive\Documents\mywork-repo\data\output\dump_samco_22_12.json"
with open(path, "r", encoding="utf-8") as f:
    data = json.load(f)


In [30]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234",
    database="company_db"
)
cursor = conn.cursor()

def get_or_create_amc(amc_name):
    cursor.execute("SELECT id FROM amcs WHERE amc_name = %s", (amc_name,))
    result = cursor.fetchone()
    if result:
        return result[0]
    cursor.execute("INSERT INTO amcs (amc_name) VALUES (%s)", (amc_name,))
    conn.commit()
    return cursor.lastrowid


In [ ]:
for scheme_name, details in data.items():
    amc_id = get_or_create_amc(details["amc_name"])

    #mutual fund data
    cursor.execute("""
        INSERT INTO mutual_funds (amc_id, benchmark_index, main_scheme_name, mutual_fund_name, monthly_aaum_date, monthly_aaum_value, scheme_launch_date, min_addl_amt, min_addl_amt_multiple, min_amt, min_amt_multiple) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        amc_id,  
        details["benchmark_index"],  
        details["main_scheme_name"],  
        details["mutual_fund_name"],  
        details["monthly_aaum_date"],  
        details["monthly_aaum_value"],  
        details["scheme_launch_date"],  
        details["min_addl_amt"],  
        details["min_addl_amt_multiple"],  
        details["min_amt"],  
        details["min_amt_multiple"]))

    mutual_fund_id = cursor.lastrowid

    #fund manager data
    for manager in details.get("fund_manager", []):
        cursor.execute("""
            INSERT INTO fund_managers (amc_id, mutual_fund_id, main_scheme_name, manager_name, qualification, managing_fund_since, total_exp) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """, (
            amc_id, 
            mutual_fund_id, 
            details["main_scheme_name"], 
            manager.get("name", ""), 
            manager.get("qualification", ""), 
            manager.get("managing_fund_since", ""), 
            manager.get("total_exp", "")))


    #load data
    entry_load = ""
    exit_load = ""

    for load in details.get("load", []):
        if load.get("name") == "entry_load":
            entry_load = load.get("value", "")
        elif load.get("name") == "exit_load":
            exit_load = load.get("value", "")

    cursor.execute("""
        INSERT INTO transformed_loads (amc_id, mutual_fund_id, main_scheme_name, entry_load, exit_load)
        VALUES (%s ,%s, %s, %s, %s)
    """, (   
            amc_id,
            mutual_fund_id,
            details["main_scheme_name"],
            entry_load,  
            exit_load))
    
    #metric data      
    for metric in details.get("metrics", []):
        cursor.execute("""
            INSERT INTO metrics (amc_id,mutual_fund_id,main_scheme_name ,name, value)
            VALUES (%s, %s, %s, %s, %s)
        """,
            (amc_id,
             mutual_fund_id,
             details["main_scheme_name"],
             metric.get("name",""),
             metric.get("value","")))

conn.commit()
cursor.close()
conn.close()

print("Data inserted successfully!")

In [28]:
conn.commit()
cursor.close()
conn.close()


In [ ]:
# for load_type, load_value in details.get("load", {}).items():
#     cursor.execute("""
#         INSERT INTO loads (mutual_fund_id, load_type, load_value)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, load_type, load_value))
    
# for metric_name, metric_value in details.get("metrics", {}).items():
#     cursor.execute("""
#         INSERT INTO metrics (mutual_fund_id, name, value)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, metric_name, metric_value))


# min_amt = details.get("min_amt", {})
# if min_amt:
#     cursor.execute("""
#         INSERT INTO fund_minimum_amounts (mutual_fund_id, amt, thraftr)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, min_amt.get("amt", "0"), min_amt.get("thraftr", "0")))

# min_addl_amt = details.get("min_addl_amt", {})
# if min_addl_amt:
#     cursor.execute("""
#         INSERT INTO fund_additional_amounts (mutual_fund_id, amt, thraftr)
#         VALUES (%s, %s, %s)""", 
#         (mutual_fund_id, min_addl_amt.get("amt", "0"), min_addl_amt.get("thraftr", "0")))